In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Directories for the dataset
train_dir = 'skin.disease/skin.disease.split/train'
valid_dir = 'skin.disease/skin.disease.split/valid'
test_dir = 'skin.disease/skin.disease.split/test'

# Parameters
img_size = (224, 224)  # Input image size
batch_size = 32
num_classes = 4  # acne, atopic, bcc, melanoma
epochs = 30  # Increase as necessary

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

valid_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Load images from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

# Define the CNN model
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth convolutional layer
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the convolutional layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization

# Output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=valid_generator.samples // batch_size,
)

# Print validation accuracy after initial training
print(f"Validation accuracy after initial training: {max(history.history['val_accuracy']) * 100:.2f}%")

# Fine-tuning phase (if you decide to fine-tune)
# Unfreeze the last few layers of the base model
for layer in model.layers[-20:]:  # Adjust based on your architecture
    layer.trainable = True

# Compile the model again with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training with fine-tuning
fine_tune_epochs = 10  # Number of epochs for fine-tuning
history_fine_tune = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=fine_tune_epochs,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=valid_generator.samples // batch_size,
)

# Print validation accuracy after fine-tuning
print(f"Validation accuracy after fine-tuning: {max(history_fine_tune.history['val_accuracy']) * 100:.2f}%")

# Evaluate on test set
test_generator = valid_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.2f}")

# Save the model
model.save('skin_disease_classifier_cnn.h5')


Found 6377 images belonging to 4 classes.
Found 288 images belonging to 4 classes.


C:\Users\azcos5\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


C:\Users\azcos5\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


199/199 ━━━━━━━━━━━━━━━━━━━━ 119s 574ms/step - accuracy: 0.5169 - loss: 1.0411 - val_accuracy: 0.5451 - val_loss: 1.0163
Epoch 2/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 207us/step - accuracy: 0.5312 - loss: 0.8705  
Epoch 3/30


C:\Users\azcos5\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


199/199 ━━━━━━━━━━━━━━━━━━━━ 116s 570ms/step - accuracy: 0.6293 - loss: 0.7838 - val_accuracy: 0.6354 - val_loss: 0.7926
Epoch 4/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 158us/step - accuracy: 0.5938 - loss: 0.7030  
Epoch 5/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 116s 570ms/step - accuracy: 0.6285 - loss: 0.7664 - val_accuracy: 0.6007 - val_loss: 0.8019
Epoch 6/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step - accuracy: 0.7188 - loss: 0.6870  
Epoch 7/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 116s 569ms/step - accuracy: 0.6451 - loss: 0.7196 - val_accuracy: 0.6285 - val_loss: 0.7428
Epoch 8/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step - accuracy: 0.6250 - loss: 0.8090  
Epoch 9/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 116s 570ms/step - accuracy: 0.6680 - loss: 0.7080 - val_accuracy: 0.6285 - val_loss: 0.7126
Epoch 10/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 0s 148us/step - accuracy: 0.7812 - loss: 0.5354  
Epoch 11/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 116s 570ms/step - accuracy: 0.6851 - loss: 0.6711 - val_accuracy: 0.6528 - val_lo

Test accuracy: 0.74
